# IBM Qiskit v2.x Certification — Section 1 Auto-Grader Notebook
## Section 1: Perform quantum operations (16%)

**Focus tasks**
- Define Pauli Operators  
- Apply quantum operations  

This notebook provides:
- Auto-graded exam-style MCQs
- Step-by-step explanations
- Optional Qiskit verification cells
- Qiskit **v2.x-only syntax**


In [ ]:
# ============================================================
# INSTALL FULL REQUIREMENTS FOR QISKIT v2.x EXAM PRACTICE
# ============================================================
# If you're running in Google Colab, you may need to restart the runtime after install.

!pip -q install "qiskit>=2.0.0" qiskit-aer numpy matplotlib sympy pylatexenc ipywidgets

import qiskit
print("✅ Qiskit version:", qiskit.__version__)
print("✅ Install complete. If widgets fail to render, restart kernel/runtime.")


In [ ]:
import numpy as np
from math import sqrt, pi

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator
from qiskit_aer import AerSimulator

import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown


## Qiskit v2.x Syntax Reinforcement (Exam-aligned)

✅ Use `QuantumCircuit` to build circuits  
✅ Use `Statevector.from_instruction(qc)` for statevector simulation  
✅ Use `Operator(gate_or_circuit).data` for matrix extraction  

✅ Shot-based simulation with AerSimulator:

```python
sim = AerSimulator()
result = sim.run(qc, shots=1024).result()
counts = result.get_counts()
```

✅ Avoid legacy `execute()` patterns from older Qiskit tutorials.

✅ Measurements:
- Use `qc = QuantumCircuit(n_qubits, n_clbits)` for convenience
- Use `qc.measure(q, c)` (or `qc.measure([..],[..])`) after building the circuit


In [ ]:
QUESTION_BANK = [
    {
        "id": "Q1",
        "topic": "Pauli",
        "question": "Which matrix corresponds to the Pauli-X operator?",
        "options": {
            "A": "[[1,0],[0,1]]",
            "B": "[[0,1],[1,0]]",
            "C": "[[1,0],[0,-1]]",
            "D": "[[0,-i],[i,0]]"
        },
        "answer": "B",
        "explanation": "Pauli-X swaps |0⟩ and |1⟩ → [[0,1],[1,0]].",
        "verify": "Use Operator(XGate()).data"
    },
    {
        "id": "Q2",
        "topic": "Pauli",
        "question": "What is the result of applying Z to |1⟩?",
        "options": {"A": "|0⟩","B": "|1⟩","C": "-|1⟩","D": "i|1⟩"},
        "answer": "C",
        "explanation": "Z applies a phase flip: Z|1⟩ = -|1⟩.",
        "verify": "Statevector.from_label('1').evolve(qc.z(0))"
    },
    {
        "id": "Q3",
        "topic": "Pauli",
        "question": "Which Pauli operator performs a bit flip?",
        "options": {"A": "X","B": "Y","C": "Z","D": "I"},
        "answer": "A",
        "explanation": "X flips |0⟩ ↔ |1⟩.",
        "verify": "Apply qc.x(0) to |0⟩"
    },
    {
        "id": "Q4",
        "topic": "Pauli",
        "question": "Which Pauli operator performs a phase flip?",
        "options": {"A": "X","B": "Y","C": "Z","D": "I"},
        "answer": "C",
        "explanation": "Z flips the phase of |1⟩ only.",
        "verify": "Apply qc.z(0) to |1⟩"
    },
    {
        "id": "Q5",
        "topic": "Pauli",
        "question": "Which operator applies both a bit flip and phase flip (up to a global phase)?",
        "options": {"A": "X","B": "Y","C": "Z","D": "H"},
        "answer": "B",
        "explanation": "Y ≈ iXZ: it flips and adds phase with imaginary factor.",
        "verify": "Operator(YGate()).data"
    },
    {
        "id": "Q6",
        "topic": "Pauli",
        "question": "Which pair anticommutes?",
        "options": {"A": "X and X","B": "I and Z","C": "X and Z","D": "Z and Z"},
        "answer": "C",
        "explanation": "XZ = -ZX → anticommute.",
        "verify": "Compare X@Z and Z@X"
    },
    {
        "id": "Q7",
        "topic": "Pauli",
        "question": "What is X|0⟩?",
        "options": {"A": "|0⟩","B": "|1⟩","C": "-|0⟩","D": "i|1⟩"},
        "answer": "B",
        "explanation": "X flips |0⟩ to |1⟩.",
        "verify": "Statevector.from_label('0').evolve(qc.x(0))"
    },
    {
        "id": "Q8",
        "topic": "Pauli",
        "question": "What is Y|0⟩?",
        "options": {"A": "|1⟩","B": "-|1⟩","C": "i|1⟩","D": "-i|1⟩"},
        "answer": "C",
        "explanation": "Y|0⟩ = i|1⟩ from matrix action.",
        "verify": "Statevector evolution"
    },
    {
        "id": "Q9",
        "topic": "Pauli",
        "question": "Which operator corresponds to the identity matrix?",
        "options": {"A": "X","B": "Y","C": "Z","D": "I"},
        "answer": "D",
        "explanation": "I leaves all states unchanged.",
        "verify": "Operator(IGate()).data"
    },
    {
        "id": "Q10",
        "topic": "Pauli",
        "question": "Which statement is true?",
        "options": {
            "A": "X introduces a phase shift on |1⟩",
            "B": "Z swaps |0⟩ and |1⟩",
            "C": "Z applies a phase shift to |1⟩",
            "D": "Y is real-valued"
        },
        "answer": "C",
        "explanation": "Z multiplies |1⟩ by -1 (phase flip).",
        "verify": "Z matrix is diagonal with -1"
    },
    {
        "id": "Q11",
        "topic": "Pauli",
        "question": "Which Pauli operator is diagonal?",
        "options": {"A": "X","B": "Y","C": "Z","D": "None"},
        "answer": "C",
        "explanation": "Z = [[1,0],[0,-1]] is diagonal.",
        "verify": "Operator(ZGate()).data"
    },
    {
        "id": "Q12",
        "topic": "Pauli",
        "question": "Which Pauli operator has purely imaginary off-diagonal terms?",
        "options": {"A": "X","B": "Y","C": "Z","D": "I"},
        "answer": "B",
        "explanation": "Y contains ±i off-diagonal elements.",
        "verify": "Y matrix has i and -i"
    },
    {
        "id": "Q13",
        "topic": "Ops",
        "question": "Starting from |0⟩, apply H then measure. What is expected?",
        "options": {"A": "100% 0","B": "100% 1","C": "~50% 0 and ~50% 1","D": "~75% 0 and ~25% 1"},
        "answer": "C",
        "explanation": "H|0⟩ = (|0⟩ + |1⟩)/√2 so equal probabilities.",
        "verify": "Simulate 2000 shots in AerSimulator"
    },
    {
        "id": "Q14",
        "topic": "Ops",
        "question": "Starting from |0⟩, apply X then H. Final state?",
        "options": {"A": "|0⟩","B": "|1⟩","C": "(|0⟩ + |1⟩)/√2","D": "(|0⟩ - |1⟩)/√2"},
        "answer": "D",
        "explanation": "X|0⟩ = |1⟩, then H|1⟩ = (|0⟩ - |1⟩)/√2.",
        "verify": "Statevector.from_instruction(qc)"
    },
    {
        "id": "Q15",
        "topic": "Ops",
        "question": "Starting from |00⟩, apply H(0) then CX(0,1). What is produced?",
        "options": {"A": "|00⟩","B": "(|00⟩ + |11⟩)/√2","C": "(|01⟩ + |10⟩)/√2","D": "|11⟩"},
        "answer": "B",
        "explanation": "This creates a Bell state with correlated outcomes.",
        "verify": "Statevector check"
    },
    {
        "id": "Q16",
        "topic": "Ops",
        "question": "What does a CX gate do?",
        "options": {"A": "Always flips target","B": "Flips control if target is 1","C": "Flips target if control is 1","D": "Swaps qubits"},
        "answer": "C",
        "explanation": "CNOT flips the target only when control qubit is |1⟩.",
        "verify": "Apply to |10⟩ gives |11⟩"
    },
    {
        "id": "Q17",
        "topic": "Ops",
        "question": "Starting from |0⟩, apply H then Z. What is the state?",
        "options": {"A": "(|0⟩ + |1⟩)/√2","B": "(|0⟩ - |1⟩)/√2","C": "|1⟩","D": "|0⟩"},
        "answer": "B",
        "explanation": "Z flips phase of |1⟩ component in superposition.",
        "verify": "Statevector evolution"
    },
    {
        "id": "Q18",
        "topic": "Ops",
        "question": "Does global phase affect measurement probabilities?",
        "options": {"A": "Yes always","B": "Yes sometimes","C": "No","D": "Only for Z"},
        "answer": "C",
        "explanation": "Global phase does not affect probabilities.",
        "verify": "Compare e^{iθ}|ψ⟩ measurements"
    },
    {
        "id": "Q19",
        "topic": "Ops",
        "question": "Which line applies Pauli-Z on qubit 2?",
        "options": {"A": "qc.Z(2)","B": "qc.apply_z(2)","C": "qc.z(2)","D": "qc.pauli_z(2)"},
        "answer": "C",
        "explanation": "Qiskit gate methods are lowercase: x(), y(), z(), h(), cx(), etc.",
        "verify": "Use qc.z(2)"
    },
    {
        "id": "Q20",
        "topic": "Ops",
        "question": "Which gate is equivalent to Rx(π) up to a global phase?",
        "options": {"A": "X","B": "H","C": "Z","D": "CX"},
        "answer": "A",
        "explanation": "X is a π rotation around X-axis.",
        "verify": "Compare Operator(X) and Operator(RXGate(pi))"
    },
    {
        "id": "Q21",
        "topic": "Ops",
        "question": "Which statement is correct?",
        "options": {"A": "H and X always commute","B": "Z and X commute","C": "Order of gates can matter","D": "All single-qubit gates commute"},
        "answer": "C",
        "explanation": "Many gates do NOT commute; order matters.",
        "verify": "Compare H∘X vs X∘H"
    },
    {
        "id": "Q22",
        "topic": "Ops",
        "question": "If you measure the Bell state (|00⟩+|11⟩)/√2, what outcomes appear?",
        "options": {"A": "00 only","B": "11 only","C": "00 and 11 only (~50/50)","D": "All four equally"},
        "answer": "C",
        "explanation": "Bell pair measurement yields correlated results only: 00 or 11.",
        "verify": "Simulate shots from Bell circuit"
    },
    {
        "id": "Q23",
        "topic": "Ops",
        "question": "Which object is used to build circuits in Qiskit?",
        "options": {"A": "QuantumGate","B": "QuantumCircuit","C": "QASMBuilder","D": "GateSystem"},
        "answer": "B",
        "explanation": "QuantumCircuit is the primary circuit builder.",
        "verify": "QuantumCircuit(n_qubits, n_clbits)"
    },
    {
        "id": "Q24",
        "topic": "Ops",
        "question": "What is the primary effect of applying H to |0⟩?",
        "options": {"A": "Flips to |1⟩","B": "Creates superposition","C": "Applies phase -1","D": "Creates entanglement"},
        "answer": "B",
        "explanation": "Hadamard creates superposition: (|0⟩+|1⟩)/√2.",
        "verify": "Statevector"
    },
    {
        "id": "Q25",
        "topic": "Ops",
        "question": "Starting from |0⟩, apply X then measure. What is the result?",
        "options": {"A": "Always 0","B": "Always 1","C": "50/50","D": "Depends on shots"},
        "answer": "B",
        "explanation": "X|0⟩ = |1⟩, measurement always gives 1.",
        "verify": "Simulate counts"
    }
]
print("✅ Loaded questions:", len(QUESTION_BANK))


In [ ]:
def build_quiz(question_bank):
    answer_widgets = {}
    quiz_items = []
    for q in question_bank:
        q_id = q["id"]
        q_text = f"**{q_id} ({q['topic']})** — {q['question']}"
        quiz_items.append(Markdown(q_text))
        opts = [f"{k}) {v}" for k, v in q["options"].items()]
        w = widgets.RadioButtons(options=opts, layout=widgets.Layout(width="100%"))
        answer_widgets[q_id] = w
        quiz_items.append(w)
        quiz_items.append(widgets.HTML("<hr>"))

    submit_btn = widgets.Button(description="✅ Submit & Grade", button_style="success")
    reset_btn = widgets.Button(description="🔄 Reset", button_style="")
    results_out = widgets.Output()

    def grade(_):
        with results_out:
            clear_output()
            score = 0
            missed = []
            for q in question_bank:
                q_id = q["id"]
                selected = answer_widgets[q_id].value
                if selected is None:
                    missed.append((q, None))
                    continue
                selected_letter = selected.split(")")[0].strip()
                if selected_letter == q["answer"]:
                    score += 1
                else:
                    missed.append((q, selected_letter))
            total = len(question_bank)
            pct = round(score/total*100, 1)
            display(Markdown(f"## ✅ Score: {score}/{total} ({pct}%)"))
            if missed:
                display(Markdown("### ❌ Review — Incorrect / Skipped Questions"))
                for q, chosen in missed:
                    chosen_str = chosen if chosen is not None else "*No answer selected*"
                    display(Markdown(
                        f"**{q['id']}** — {q['question']}\n\n"
                        f"- Your answer: **{chosen_str}**\n"
                        f"- Correct answer: **{q['answer']}**\n\n"
                        f"✅ Explanation: {q['explanation']}\n\n"
                        f"🧪 Verification hint: `{q['verify']}`\n"
                        f"---"
                    ))
            else:
                display(Markdown("🎉 Perfect score — great job!"))

    def reset(_):
        for w in answer_widgets.values():
            w.value = None
        with results_out:
            clear_output()
            display(Markdown("✅ Reset complete — you can retake the quiz."))

    submit_btn.on_click(grade)
    reset_btn.on_click(reset)

    display(Markdown("# Section 1 Auto-Graded Quiz"))
    for item in quiz_items:
        display(item)

    display(widgets.HBox([submit_btn, reset_btn]))
    display(results_out)

build_quiz(QUESTION_BANK)


## Optional: Qiskit v2.x Verification Cells

In [ ]:
from qiskit.circuit.library import IGate, XGate, YGate, ZGate

pauli_mats = {
    "I": Operator(IGate()).data,
    "X": Operator(XGate()).data,
    "Y": Operator(YGate()).data,
    "Z": Operator(ZGate()).data
}

for k, v in pauli_mats.items():
    print(f"Pauli-{k}:\n{v}\n")


In [ ]:
qc = QuantumCircuit(1)
qc.h(0)
sv = Statevector.from_instruction(qc)
print("Statevector after H on |0⟩:\n", np.round(sv.data, 3))


In [ ]:
qc = QuantumCircuit(1, 1)
qc.h(0)
qc.measure(0, 0)

sim = AerSimulator()
result = sim.run(qc, shots=2000).result()
counts = result.get_counts()
print("Counts:", counts)


In [ ]:
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

sim = AerSimulator()
result = sim.run(qc, shots=2000).result()
counts = result.get_counts()
print("Bell counts:", counts)
